In [ ]:
import pandas as pd

# 광어

In [ ]:
# 해면어업 광어 생산량
ff_production = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/2조/이경희/행정구역별 해면어업 넙치류 생산량.csv')

In [ ]:
ff_production.isna().sum().sum() # 결측치 확인

0

In [ ]:
ff_production['행정구역별'].values

array(['부산광역시', '인천광역시', '울산광역시', '경기도', '강원도', '충청남도', '전라북도', '전라남도',
       '경상북도', '경상남도', '제주도'], dtype=object)

In [ ]:
# 컬럼명 재정의 
for y in range(3):
  for m in range(1,10) :
    old_col = ff_production.columns[12*y+m]
    new_col = ff_production.columns[12*y+m].replace(" ","").replace(f"{2019+y}.0{m}월",f"{2019+y}.{m}") 
    ff_production.rename(columns = {old_col:new_col}, inplace =True) 
# ff_production.columns[1].replace(" ","").replace(f"{2018+1}.0{str(1)}월",f"{2018+1}년.{str(1)}") 
  for m in range(10,13):
    old_col = ff_production.columns[12*y+m]
    new_col = ff_production.columns[12*y+m].replace(" ","").replace("월","").replace("년","")
    ff_production.rename(columns = {old_col:new_col}, inplace =True) 

1. 생산량data의 행정구역이 총 11개로 구분되어있고, 우리가 보고자 하는 곳에서 경기도와 제주도에 해당되는 곳 없음   
  → '경기도'와 '제주도' row 제거  
2. 평균가 data에서 각 행정구역에 해당하는 지역별로 합쳐서 평균내어 계산 
  → 그럼 생산량 data와 같은 형태로 9개의 행정구역으로 구분됨  
3. 생산량 data가 월별로 나타나 있으니까 평균가 data도 월별로 평균내서 생산량 data와 형식 맞춤

4. 준비된 데이터로 관계분석  

---------------------------------------------------------------------------
* 부산광역시 - '부산시'
* 인천광역시 - '인천'
* 울산광역시 - '울산' 
* 경기도 - 
* 강원도 -  '삼척', '강릉시', '속초시', '동해시'
* 충청남도 - '서산', '안면도', '대천서부', '서천서부',
* 전라북도 - '군산시',
* 전라남도 - '목포', '전남서부어류양식', '고흥군', '여수',
* 경상북도 - '포항',
* 경상남도 - '남해군', '통영', '거제',
* 제주도 - 

생산량 data에서 '경기도'와 '제주도' row 제거  

In [ ]:
ff_production.drop([3,10], axis=0, inplace=True)
ff_production.reset_index(drop=True, inplace=True)

 평균가 data에서 각 행정구역에 해당하는 지역별 월별 평균냄   
  → 그럼 생산량 data와 같은 형태로 9개의 행정구역으로 구분됨  

In [ ]:
# 우선 광어의 생산량과 가격을 비교할 것이기 때문에 광어만 뽑아서 저장함
df = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/2조/이경희/수산물_위판정보_전처리완료.csv')
ff_df = df[df['수산물표준코드명'] == '광어']
ff_df.reset_index(drop=True, inplace=True)

In [ ]:
# 월별 평균가를 넣은 df형식을 생산량df 형태와 동일하도록 복사해옴
from copy import deepcopy
ff_price = deepcopy(ff_production)
ff_price.set_index('행정구역별',inplace=True)

# ff_price 의 모든 값을 0으로 초기화
for row in range(len(ff_price)):
  for col in range(len(ff_price.columns)):
    ff_price.iloc[row,col] = 0

In [ ]:
# 위판일자 값 수정
for row in range(len(ff_df)):
  ff_df.loc[row, '위판일자'] = ff_df.loc[row, '위판일자'].replace('-0','-')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


이것도 
ex)  ff_df를 보면 분명 2019년 3월 강원도에 속하는 지역들의 평균가 값들이 많이 나와있지만   
ff_price 데이터프레임에 2019.3월 강원도의 값이 대체가 안 되어 있음


In [ ]:
# 각 행정구역에 해당하는 지역별로 평균내기
for y in range(3):  
  for m in range(1,13) :  
    Busan, Incheon, Ulsan, Gangwon, Chungnam, Jeonbuk, Jeonnam, Gyeongbuk, Gyeongnam = [], [], [], [], [], [], [], [], []
    for d in range(1,32): 
      for row in range(len(ff_df)):
        if ff_df.loc[row,'산지조합명'] == '부산시' and ff_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Busan.append(ff_df.loc[row,'평균가'])
        elif ff_df.loc[row,'산지조합명'] == '인천' and ff_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Incheon.append(ff_df.loc[row,'평균가'])
        elif ff_df.loc[row,'산지조합명'] == '울산' and ff_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Ulsan.append(ff_df.loc[row,'평균가'])
        elif (ff_df.loc[row,'산지조합명'] == '삼척' or ff_df.loc[row,'산지조합명'] == '강릉시' or ff_df.loc[row,'산지조합명'] == '속초시' or ff_df.loc[row,'산지조합명'] == '동해시') and ff_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Gangwon.append(ff_df.loc[row,'평균가'])
        elif (ff_df.loc[row,'산지조합명'] == '서산' or ff_df.loc[row,'산지조합명'] == '안면도' or ff_df.loc[row,'산지조합명'] == '대천서부' or ff_df.loc[row,'산지조합명'] == '서천서부') and ff_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Chungnam.append(ff_df.loc[row,'평균가'])
        elif ff_df.loc[row,'산지조합명'] == '군산시' and ff_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Jeonbuk.append(ff_df.loc[row,'평균가'])
        elif (ff_df.loc[row,'산지조합명'] == '삼척' or ff_df.loc[row,'산지조합명'] == '목포' or ff_df.loc[row,'산지조합명'] == '전남서부어류양식' or ff_df.loc[row,'산지조합명'] == '고흥군' or ff_df.loc[row,'산지조합명'] == '여수') and ff_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Jeonnam.append(ff_df.loc[row,'평균가'])
        elif ff_df.loc[row,'산지조합명'] == '포항' and ff_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Gyeongbuk.append(ff_df.loc[row,'평균가'])
        elif (ff_df.loc[row,'산지조합명'] == '남해군' or ff_df.loc[row,'산지조합명'] == '통영' or ff_df.loc[row,'산지조합명'] == '거제') and ff_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Gyeongnam.append(ff_df.loc[row,'평균가'])
    try : ff_price.loc['부산광역시',f'{2019+y}.{m}'] = round(sum(Busan)/len(Busan), 1)
    except : pass
    try : ff_price.loc['인천광역시',f'{2019+y}.{m}'] = round(sum(Incheon)/len(Incheon), 1)
    except : pass
    try : ff_price.loc['울산광역시',f'{2019+y}.{m}'] = round(sum(Ulsan)/len(Ulsan), 1)
    except : pass
    try : ff_price.loc['강원도',f'{2019+y}.{m}'] = round(sum(Gangwon)/len(Gangwon), 1)
    except : pass
    try : ff_price.loc['충청남도',f'{2019+y}.{m}'] = round(sum(Chungnam)/len(Chungnam), 1)
    except : pass
    try : ff_price.loc['전라북도',f'{2019+y}.{m}'] = round(sum(Jeonbuk)/len(Jeonbuk), 1)
    except : pass
    try : ff_price.loc['전라남도',f'{2019+y}.{m}'] = round(sum(Jeonnam)/len(Jeonnam), 1)
    except : pass
    try : ff_price.loc['경상북도',f'{2019+y}.{m}'] = round(sum(Gyeongbuk)/len(Gyeongbuk), 1)
    except : pass
    try : ff_price.loc['경상남도',f'{2019+y}.{m}'] = round(sum(Gyeongnam)/len(Gyeongnam), 1)
    except : pass

In [ ]:
# 행정구역별_월별_광어_평균가
# pd.DataFrame(ff_price).to_csv('/content/drive/MyDrive/멀티캠퍼스/2조/이경희/상관관계 분석 data/행정구역별_평균가_광어(해면어업).csv', index=False)

In [ ]:
# ff_price  

# 방어

In [ ]:
# 해면어업 방어 생산량 
yellowtail_production = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/2조/이경희/행정구역별 해면어업 방어류 생산량.csv')

In [ ]:
yellowtail_production.isna().sum().sum() # 결측치 확인

0

In [ ]:
yellowtail_production['행정구역별'].values

array(['부산광역시', '인천광역시', '울산광역시', '강원도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주도'], dtype=object)

In [ ]:
# 컬럼명 재정의 
for y in range(3):
  for m in range(1,10) :
    old_col = yellowtail_production.columns[12*y+m]
    new_col = yellowtail_production.columns[12*y+m].replace(" ","").replace(f"{2019+y}.0{m}월",f"{2019+y}.{m}") 
    yellowtail_production.rename(columns = {old_col:new_col}, inplace =True) 
# yellowtail_production.columns[1].replace(" ","").replace(f"{2018+1}.0{str(1)}월",f"{2018+1}년.{str(1)}") 
  for m in range(10,13):
    old_col = yellowtail_production.columns[12*y+m]
    new_col = yellowtail_production.columns[12*y+m].replace(" ","").replace("월","").replace("년","")
    yellowtail_production.rename(columns = {old_col:new_col}, inplace =True) 

In [ ]:
# yellowtail_production

생산량 data에서 '경기도'와 '제주도' row 제거  

* 생산량data의 행정구역이 총 1개로 구분되어있고, 우리가 보고자 하는 곳에서 제주도에 해당되는 곳 없음  
→ '제주도' row 제거

In [ ]:
yellowtail_production.drop([9], axis=0, inplace=True)

In [ ]:
len(yellowtail_production['행정구역별'].values)

9

평균가 data에서 각 행정구역에 해당하는 지역별 월별 평균냄   
  → 그럼 생산량 data와 같은 형태로 9개의 행정구역으로 구분됨  

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/2조/이경희/수산물_위판정보_전처리완료.csv')
yellowtail_df = df[df['수산물표준코드명'] == '방어']
yellowtail_df.reset_index(drop=True, inplace=True)

In [ ]:
# 월별 평균가를 넣은 df형식을 생산량df 형태와 동일하도록 복사해옴
from copy import deepcopy
yellowtail_price = deepcopy(yellowtail_production)
yellowtail_price.set_index('행정구역별',inplace=True)

# yellowtail_price 의 모든 값을 0으로 초기화
for row in range(len(yellowtail_price)):
  for col in range(len(yellowtail_price.columns)):
    yellowtail_price.iloc[row,col] = 0

In [ ]:
for row in range(len(yellowtail_df)):
  yellowtail_df.loc[row, '위판일자'] = yellowtail_df.loc[row, '위판일자'].replace('-0','-')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
# 각 행정구역에 해당하는 지역별로 평균내기
for y in range(3): 
  for m in range(1,13) :   
    Busan, Incheon, Ulsan, Gangwon, Chungnam, Jeonbuk, Jeonnam, Gyeongbuk, Gyeongnam = [], [], [], [], [], [], [], [], []
    for d in range(1,32): 
      for row in range(len(yellowtail_df)):
        if yellowtail_df.loc[row,'산지조합명'] == '부산시' and yellowtail_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Busan.append(yellowtail_df.loc[row,'평균가'])
        if yellowtail_df.loc[row,'산지조합명'] == '인천' and yellowtail_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Incheon.append(yellowtail_df.loc[row,'평균가'])
        if yellowtail_df.loc[row,'산지조합명'] == '울산' and yellowtail_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Ulsan.append(yellowtail_df.loc[row,'평균가'])
        if (yellowtail_df.loc[row,'산지조합명'] == '삼척' or yellowtail_df.loc[row,'산지조합명'] == '강릉시' or yellowtail_df.loc[row,'산지조합명'] == '속초시' or yellowtail_df.loc[row,'산지조합명'] == '동해시') and yellowtail_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Gangwon.append(yellowtail_df.loc[row,'평균가'])
        if (yellowtail_df.loc[row,'산지조합명'] == '서산' or yellowtail_df.loc[row,'산지조합명'] == '안면도' or yellowtail_df.loc[row,'산지조합명'] == '대천서부' or yellowtail_df.loc[row,'산지조합명'] == '서천서부') and yellowtail_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Chungnam.append(yellowtail_df.loc[row,'평균가'])
        if yellowtail_df.loc[row,'산지조합명'] == '군산시' and yellowtail_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Jeonbuk.append(yellowtail_df.loc[row,'평균가'])
        if (yellowtail_df.loc[row,'산지조합명'] == '삼척' or yellowtail_df.loc[row,'산지조합명'] == '목포' or yellowtail_df.loc[row,'산지조합명'] == '전남서부어류양식' or yellowtail_df.loc[row,'산지조합명'] == '고흥군' or yellowtail_df.loc[row,'산지조합명'] == '여수') and yellowtail_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Jeonnam.append(yellowtail_df.loc[row,'평균가'])
        if yellowtail_df.loc[row,'산지조합명'] == '포항' and yellowtail_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Gyeongbuk.append(yellowtail_df.loc[row,'평균가'])
        if (yellowtail_df.loc[row,'산지조합명'] == '남해군' or yellowtail_df.loc[row,'산지조합명'] == '통영' or yellowtail_df.loc[row,'산지조합명'] == '거제') and yellowtail_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Gyeongnam.append(yellowtail_df.loc[row,'평균가']  
    try : yellowtail_price.loc['부산광역시',f'{2019+y}.{m}'] = round(sum(Busan)/len(Busan), 2)
    except : pass
    try : yellowtail_price.loc['인천광역시',f'{2019+y}.{m}'] = round(sum(Incheon)/len(Incheon), 2)
    except : pass
    try : yellowtail_price.loc['울산광역시',f'{2019+y}.{m}'] = round(sum(Ulsan)/len(Ulsan), 2)
    except : pass
    try : yellowtail_price.loc['강원도',f'{2019+y}.{m}'] = round(sum(Gangwon)/len(Gangwon), 2)
    except : pass
    try : yellowtail_price.loc['충청남도',f'{2019+y}.{m}'] = round(sum(Chungnam)/len(Chungnam), 2)
    except : pass
    try : yellowtail_price.loc['전라북도',f'{2019+y}.{m}'] = round(sum(Jeonbuk)/len(Jeonbuk), 2)
    except : pass
    try : yellowtail_price.loc['전라남도',f'{2019+y}.{m}'] = round(sum(Jeonnam)/len(Jeonnam), 2)
    except : pass
    try : yellowtail_price.loc['경상북도',f'{2019+y}.{m}'] = round(sum(Gyeongbuk)/len(Gyeongbuk), 2)
    except : pass
    try : yellowtail_price.loc['경상남도',f'{2019+y}.{m}'] = round(sum(Gyeongnam)/len(Gyeongnam), 2)
    except : pass

In [ ]:
# 행정구역별_월별_방어_평균가
# pd.DataFrame(yellowtail_price).to_csv('/content/drive/MyDrive/멀티캠퍼스/2조/이경희/상관관계 분석 data/행정구역별_평균가_방어(해면어업).csv',index=False)

In [ ]:
# yellowtail_price

# 감성돔

In [ ]:
# 해면어업 감성돔 생산량 
black_porgy_production = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/2조/이경희/행정구역별 해면어업 감성돔 생산량.csv')

In [ ]:
black_porgy_production.isna().sum().sum()

0

In [ ]:
black_porgy_production['행정구역별'].values

array(['부산광역시', '인천광역시', '울산광역시', '경기도', '강원도', '충청남도', '전라북도', '전라남도',
       '경상북도', '경상남도', '제주도'], dtype=object)

In [ ]:
# 컬럼명 재정의 
for y in range(3):
  for m in range(1,10) :
    old_col = black_porgy_production.columns[12*y+m]
    new_col = black_porgy_production.columns[12*y+m].replace(" ","").replace(f"{2019+y}.0{m}월",f"{2019+y}.{m}") 
    black_porgy_production.rename(columns = {old_col:new_col}, inplace =True) 
# black_porgy_production.columns[1].replace(" ","").replace(f"{2018+1}.0{str(1)}월",f"{2018+1}년.{str(1)}") 
  for m in range(10,13):
    old_col = black_porgy_production.columns[12*y+m]
    new_col = black_porgy_production.columns[12*y+m].replace(" ","").replace("월","").replace("년","")
    black_porgy_production.rename(columns = {old_col:new_col}, inplace =True) 

In [ ]:
# black_porgy_production

생산량 data에서 '경기도'와 '제주도' row 제거

* 생산량data의 행정구역이 총 1개로 구분되어있고, 우리가 보고자 하는 곳에서 제주도에 해당되는 곳 없음  
→ '제주도' row 제거

In [ ]:
black_porgy_production.drop([3,10], axis=0, inplace=True)
black_porgy_production.reset_index(drop=True, inplace=True)

In [ ]:
# len(black_porgy_production['행정구역별'].values)

In [ ]:
# black_porgy_production

평균가 data에서 각 행정구역에 해당하는 지역별 월별 평균냄 
  → 그럼 생산량 data와 같은 형태로 9개의 행정구역으로 구분됨  

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/2조/이경희/수산물_위판정보_전처리완료.csv')
black_porgy_df = df[df['수산물표준코드명'] == '감성돔']
black_porgy_df.reset_index(drop=True, inplace=True)

In [ ]:
# 월별 평균가를 넣은 df형식을 생산량df 형태와 동일하도록 복사해옴
from copy import deepcopy
black_porgy_price = deepcopy(black_porgy_production)
black_porgy_price.set_index('행정구역별',inplace=True)

# black_porgy_price 의 모든 값을 0으로 초기화
for row in range(len(black_porgy_price)):
  for col in range(len(black_porgy_price.columns)):
    black_porgy_price.iloc[row,col] = 0

In [ ]:
for row in range(len(black_porgy_df)):
  black_porgy_df.loc[row, '위판일자'] = black_porgy_df.loc[row, '위판일자'].replace('-0','-')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
# 각 행정구역에 해당하는 지역별로 평균내기
for y in range(3): 
  for m in range(1,13) :   
    Busan, Incheon, Ulsan, Gangwon, Chungnam, Jeonbuk, Jeonnam, Gyeongbuk, Gyeongnam = [], [], [], [], [], [], [], [], []
    for d in range(1,32): 
      for row in range(len(black_porgy_df)):
        if black_porgy_df.loc[row,'산지조합명'] == '부산시' and black_porgy_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Busan.append(black_porgy_df.loc[row,'평균가'])
        elif black_porgy_df.loc[row,'산지조합명'] == '인천' and black_porgy_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Incheon.append(black_porgy_df.loc[row,'평균가'])
        elif black_porgy_df.loc[row,'산지조합명'] == '울산' and black_porgy_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Ulsan.append(black_porgy_df.loc[row,'평균가'])
        elif (black_porgy_df.loc[row,'산지조합명'] == '삼척' or black_porgy_df.loc[row,'산지조합명'] == '강릉시' or black_porgy_df.loc[row,'산지조합명'] == '속초시' or black_porgy_df.loc[row,'산지조합명'] == '동해시') and black_porgy_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Gangwon.append(black_porgy_df.loc[row,'평균가'])
        elif (black_porgy_df.loc[row,'산지조합명'] == '서산' or black_porgy_df.loc[row,'산지조합명'] == '안면도' or black_porgy_df.loc[row,'산지조합명'] == '대천서부' or black_porgy_df.loc[row,'산지조합명'] == '서천서부') and black_porgy_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Chungnam.append(black_porgy_df.loc[row,'평균가'])
        elif black_porgy_df.loc[row,'산지조합명'] == '군산시' and black_porgy_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Jeonbuk.append(black_porgy_df.loc[row,'평균가'])
        elif (black_porgy_df.loc[row,'산지조합명'] == '삼척' or black_porgy_df.loc[row,'산지조합명'] == '목포' or black_porgy_df.loc[row,'산지조합명'] == '전남서부어류양식' or black_porgy_df.loc[row,'산지조합명'] == '고흥군' or black_porgy_df.loc[row,'산지조합명'] == '여수') and black_porgy_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Jeonnam.append(black_porgy_df.loc[row,'평균가'])
        elif black_porgy_df.loc[row,'산지조합명'] == '포항' and black_porgy_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Gyeongbuk.append(black_porgy_df.loc[row,'평균가'])
        elif (black_porgy_df.loc[row,'산지조합명'] == '남해군' or black_porgy_df.loc[row,'산지조합명'] == '통영' or black_porgy_df.loc[row,'산지조합명'] == '거제') and black_porgy_df.loc[row,'위판일자'] == f'{2019+y}-{m}-{d}' : Gyeongnam.append(black_porgy_df.loc[row,'평균가'])
    try : black_porgy_price.loc['부산광역시',f'{2019+y}.{m}'] = round(sum(Busan)/len(Busan), 1)
    except : pass
    try : black_porgy_price.loc['인천광역시',f'{2019+y}.{m}'] = round(sum(Incheon)/len(Incheon), 1)
    except : pass
    try : black_porgy_price.loc['울산광역시',f'{2019+y}.{m}'] = round(sum(Ulsan)/len(Ulsan), 1)
    except : pass
    try : black_porgy_price.loc['강원도',f'{2019+y}.{m}'] = round(sum(Gangwon)/len(Gangwon), 1)
    except : pass
    try : black_porgy_price.loc['충청남도',f'{2019+y}.{m}'] = round(sum(Chungnam)/len(Chungnam), 1)
    except : pass
    try : black_porgy_price.loc['전라북도',f'{2019+y}.{m}'] = round(sum(Jeonbuk)/len(Jeonbuk), 1)
    except : pass
    try : black_porgy_price.loc['전라남도',f'{2019+y}.{m}'] = round(sum(Jeonnam)/len(Jeonnam), 1)
    except : pass
    try : black_porgy_price.loc['경상북도',f'{2019+y}.{m}'] = round(sum(Gyeongbuk)/len(Gyeongbuk), 1)
    except : pass
    try : black_porgy_price.loc['경상남도',f'{2019+y}.{m}'] = round(sum(Gyeongnam)/len(Gyeongnam), 1)
    except : pass

In [ ]:
# 행정구역별_월별_광어_평균가
# pd.DataFrame(black_porgy_price).to_csv('/content/drive/MyDrive/멀티캠퍼스/2조/이경희/상관관계 분석 data/행정구역별_평균가_감성돔(해면어업).csv', index=False)